In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gc

In [2]:
data = np.genfromtxt('day24_input.txt', delimiter='\n', dtype=str)

In [3]:
def flip_tiles(data, image_count):
    tiles = {}
    delta = {'e':np.array([0,1]), 'se':np.array([1,0]), 'sw':np.array([1,-1]),
             'w':np.array([0,-1]), 'nw':np.array([-1,0]), 'ne':np.array([-1,1])}
    
    for line in data:
        start = np.array([0,0])
        chars = list(line)
        
        while len(chars) > 0:
            key = ''
            c = chars.pop(0)
            key += c
            if c == 's' or c =='n':
                c = chars.pop(0)
                key += c
            start = start+delta[key]
            
        start = tuple(start)
        if start not in tiles.keys():
            tiles[start] = True
        else:
            tiles[start] = not tiles[start]
            
        image_count = plot_tiles(tiles, image_count, 'build')#, [-65.5, 66.5])
            
    return tiles, image_count

def next_day(tiles):
    new_tiles = {}
    delta = {'e':np.array([0,1]), 'se':np.array([1,0]), 'sw':np.array([1,-1]),
             'w':np.array([0,-1]), 'nw':np.array([-1,0]), 'ne':np.array([-1,1])}
    
    for key in tiles.keys():
        black = tiles[key]
        count = count_neighbours(tiles, key, delta)
        
        if black:
            if count == 1 or count == 2:
                new_tiles[key] = True
        else:
            if count == 2:
                new_tiles[key] = True
            
        for d_key in delta.keys():
            n_key = np.array(list(key))+delta[d_key]
            n_key = tuple(n_key)
            #we will do the known ones anyway, so skip
            if n_key in tiles.keys():
                continue
            
            count = count_neighbours(tiles, n_key, delta)
            #only keep the black ones
            if count == 2:
                new_tiles[n_key] = True
    
    return new_tiles

def count_neighbours(tiles, key, delta):
    key = np.array(list(key))
    count = 0
    for d_key in delta.keys():
        pos = key+delta[d_key]
        pos = tuple(pos)
        if pos in tiles.keys():
            count += tiles[pos]
            
    return count

def plot_tiles(tiles, image_count, frame_type, min_max=None):
    pos = []
    for key in tiles.keys():
        if tiles[key]:
            pos.append(list(key))
            pos[-1][0] = pos[-1][0]+(0.5*pos[-1][1])
    pos = np.array(pos)
    x = pos[:,1]
    y = pos[:,0]
    
    min_x = np.min(x)
    max_x = np.max(x)
    if max_x < 0:
        max_x = 0
    elif min_x >  0:
        min_x = 0
    
    min_y = np.min(y)
    max_y = np.max(y)
    if max_y < 0:
        max_y = 0
    elif min_y >  0:
        min_y = 0
        
    if min_max is None:
        p_min = np.min([min_x, min_y])-0.5
        p_max = np.max([max_x, max_y])+0.5
    else:
        p_min = min_max[0]
        p_max = min_max[1]
        
    xgrid = int(max_x-min_x)
    ygrid = int(max_y-min_y)
    
    plt.figure(figsize=(50,50))
    ax = plt.axes()
    ax.hexbin(x, y, gridsize=(xgrid,ygrid), mincnt=1, cmap='Greys_r')
    ax.set_xlim(p_min,p_max)
    ax.set_ylim(p_min,p_max)
    ax.axis('off')
    name = './Day24_images/'+frame_type+'_'
    if image_count < 1000:
        name += '0'
    if image_count < 100:
        name += '0'
    if image_count < 10:
        name += '0'
    name += str(image_count)
    name += '.png'
    plt.savefig(name, bbox_inches='tight')
    ax.clear()
    plt.clf()
    plt.close()
    plt.close("all")
    
    #print(p_min, p_max)
    
    image_count += 1
    return image_count

def generate_images(data, times=100):
    image_count = 0
    tiles, image_count = flip_tiles(data, image_count)
    gc.collect()
    image_count = plot_tiles(tiles, 0, 'day')#, [-65.5, 66.5])

    for i in range(0, times):
        tiles = next_day(tiles)
        image_count = plot_tiles(tiles, image_count, 'day')#, [-65.5, 66.5])
        
    gc.collect()
    return

generate_images(data)